In [ ]:
from numpy import sqrt,log,argmax
import numpy as np
from torch.distributions.bernoulli import Bernoulli
from torch import tensor
import matplotlib.pyplot as plt
from copy import deepcopy

In [ ]:
class UCB():

  def __init__(self,n_actions):

    self.n_actions = n_actions
    self.means = [0]*n_actions
    self.counts = [0]*n_actions
    self.f = lambda mu,n,t: mu +sqrt(  log(1+  t*(log(t)**2) ) / (2*n) )
    


  def update(self,action,value):

    
    self.counts[action]+=1
    n = self.counts[action] ## it's N+1 
    old_mean = self.means[action]
    new_mean = old_mean * (1 - (1/n)) + value * (1/n)

    self.means[action]= new_mean



  def pick_action(self,t):


    for action in range(self.n_actions):
      if self.counts[action]==0:return action
    
    else :

      pot_vals = [] ## potential values
      for action in range(self.n_actions):

        mu = self.means[action]
        n  = self.counts[action]
        val = self.f(mu,n,t)
        pot_vals.append(val)
    
      return argmax(pot_vals)



class bandits():

  def __init__(self,means):

    self.n_bandits = len(means)
    self.true_vals  = means
    self.bandits = [Bernoulli(tensor([mean])) for mean in means ]
    
  
  def pull(self,arm):

    return self.bandits[arm].sample().item()


In [ ]:
class KL_UCB():

  def __init__(self,n_actions):

    self.n_actions = n_actions
    self.means = [1e-1]*n_actions
    self.counts = [1]*n_actions
    
    kl = lambda p,q : p * log(p/q) + (1-p)* log( (1-p)/ (1-q))
    barrier = lambda n,t : log(1+  t*(log(t)**2) ) / n
    self.objective = lambda p,q,n,t : barrier(n,t) - kl(p,q)
    



  def update(self,action,value):

    
    self.counts[action]+=1
    n = self.counts[action] ## it's N+1 
    old_mean = self.means[action]
    #print("old mean = ",old_mean,"n+1=",n)
    new_mean = old_mean * (1 - (1/n)) + value * (1/n)
    #print("new mean =",new_mean)

    self.means[action]= new_mean
  
  def dichotomy(self,f,start):



    eps = 1e-7
    u = deepcopy(start)
    v= 1 - eps

    if f(v)>0: return 1
    if f(u) <=0: return u

    if f(u)*f(v) >=0 : 
      print("There must be an error")
    
    for i in range(10):
      mid = (u+v)/2
      if f(u) * f(mid) < 0:
          u = u 
          v = mid
      elif f(v)*f(mid) < 0:
          u = mid
          v = v
      elif f(mid) == 0:
        #print("Found exact solution")
        return mid

      else :
        print("Bisection has error")
        return None
      
    return mid
    

  def pick_action(self,t):

    for action in range(self.n_actions):
      if self.counts[action]==0:return action

    else:

      pot_vals = [] ## potential values
      
      for action in range(self.n_actions):


        mu = self.means[action]
        n  = self.counts[action]
        objt = lambda q: self.objective(mu,q,n,t)
        #print("mu=",mu,"n=",n,"t=",t)
        val = self.dichotomy(objt,start=mu)
        pot_vals.append(val)
    
      return argmax(pot_vals)

In [ ]:
class problem():

  def __init__(self,means,KL):

    self.bandits = bandits(means)
    if KL:
      self.algo = KL_UCB(self.bandits.n_bandits)
    else :
      self.algo = UCB(self.bandits.n_bandits)

    #self.algo = UCB(self.bandits.n_bandits)
    self.history = []
    self.regret = 0
    self.max = max(self.bandits.true_vals)
    self.means = self.bandits.true_vals

  def run(self,n_steps):

    

    for t in range(1,n_steps):

      act = self.algo.pick_action(t)
      ret = self.bandits.pull(act)
      #print("Picked action",act,"return is",ret)
      #print("updating with",act,ret)
      self.algo.update(act,ret)
      #print("current means=",self.algo.means)
      state = deepcopy(self.algo.means)
      self.history.append(state)
      self.regret += (self.max- self.means[act])
  
  def plot(self):

      hist= np.asarray(self.history).T

      for i,history in enumerate(hist):
        
        true_val = self.bandits.true_vals[i]
        plt.plot(history,label="Mu="+str(true_val))
        plt.legend()
      
      print("Final estimated values are", self.history[-1])
      plt.show()



def run_experiment(KL=False,n_trials=20,n_steps=10000,n_scales=30,mu=0.5):

  deltas = np.linspace(-mu,1-mu,n_scales)
  
  d_regrets = []

  for delta in deltas:

    means = [mu,mu+delta]
    regrets = []

    for i in range(n_trials):
      pb = problem(means,KL)
      pb.run(n_steps)
      regrets.append(pb.regret)

    d_mean_regret = np.mean(regrets)
    print("Delta",delta,"average regret is",d_mean_regret)
    d_regrets.append(d_mean_regret)
  
  return deltas,d_regrets

  

In [ ]:
mu = 0.5
deltas,d_regrets = run_experiment(KL=False,mu=mu)
_,d_regrets_kl = run_experiment(KL=True,mu=mu)

plt.plot(deltas,d_regrets,label='UCB')
plt.plot(deltas,d_regrets_kl,label='KL UCB')
plt.title(" mu ="+str(mu))
plt.legend()
plt.show()

Delta -0.5 average regret is 12.5
Delta -0.46551724137931033 average regret is 13.174137931034476
Delta -0.43103448275862066 average regret is 15.215517241379317
Delta -0.39655172413793105 average regret is 15.881896551724134
Delta -0.3620689655172414 average regret is 16.564655172413808
Delta -0.3275862068965517 average regret is 17.60775862068965
Delta -0.2931034482758621 average regret is 20.517241379310313
Delta -0.25862068965517243 average regret is 22.81034482758617
Delta -0.22413793103448276 average regret is 24.94655172413797
Delta -0.1896551724137931 average regret is 30.667241379310394
Delta -0.15517241379310343 average regret is 34.15344827586202
Delta -0.12068965517241381 average regret is 36.23103448275851
Delta -0.08620689655172414 average regret is 49.34482758620639
Delta -0.051724137931034475 average regret is 56.01206896551631
Delta -0.017241379310344862 average regret is 50.10431034482539
Delta 0.017241379310344862 average regret is 49.15517241379091
Delta 0.051724137

KeyboardInterrupt: ignored

In [ ]:
mu = 0.1
deltas,d_regrets = run_experiment(KL=False,mu=mu)
_,d_regrets_kl = run_experiment(KL=True,mu=mu)

plt.plot(deltas,d_regrets,label='UCB')
plt.plot(deltas,d_regrets_kl,label='KL UCB')
plt.title(" mu ="+str(mu))
plt.legend()
plt.show()

In [ ]:
mu = 0.9
deltas,d_regrets = run_experiment(KL=False,mu=mu)
_,d_regrets_kl = run_experiment(KL=True,mu=mu)

plt.plot(deltas,d_regrets,label='UCB')
plt.plot(deltas,d_regrets_kl,label='KL UCB')
plt.title(" mu ="+str(mu))
plt.legend()
plt.show()